In [16]:
# Solve the problem using k-d tree and a sorted array to hold the 10-minimum
require 'benchmark'

class Point
  CLOSEST_POINTS_LIMIT = 10
  attr_reader :x, :y, :z, :min_points
  def initialize(x:, y:, z:)
    @x = clamp(x)
    @y = clamp(y)
    @z = clamp(z)
    @min_points = []
  end
  def closest_points(points:)
    points.sort_by { |point| distance_to(point: point) }.first(CLOSEST_POINTS_LIMIT)
  end
  def cloest_points_2(node:)
    @min_points = [[nil, 3]] * 10
    searchNearest(node, 0)
    return @min_points.map{ |p| p[0] }
  end
  def searchNearest(node, axis)
    return if node.nil?
    new_value = node.value.distance_to(:point => self)
    if new_value < @min_points.last[1]
      index = @min_points.bsearch_index { |p| p[1] >= new_value}
      @min_points.insert(index, [node.value, new_value])
      @min_points.pop()
    end
    n_val, t_val = attrbyAxis(node.value, axis), attrbyAxis(self, axis)
    axis_old = axis
    axis = (axis+1) % 3
    return if node.left.nil? and node.right.nil?
    return searchNearest(node.right, axis) if node.left.nil? 
    return searchNearest(node.left, axis) if node.right.nil? 
    adv = n_val >= t_val ? node.left : node.right
    dis_adv = n_val >= t_val ? node.right : node.left
    searchNearest(adv, axis)
    searchNearest(dis_adv, axis) if @min_points.last[1] > 
    (t_val - n_val).abs ** 2
  end
  def distance_to(point:)
    #Math.sqrt((x - point.x) ** 2 + (y - point.y) ** 2 + (z - point.z) ** 2)
    (x - point.x) ** 2 + (y - point.y) ** 2 + (z - point.z) ** 2 # equivalent to above
  end
  def self.random
    new(x: rand(), y: rand(), z: rand())
  end
  private
  def clamp(coordinate)
    [[coordinate, 0.0].max, 1.0].min
  end
  def attrbyAxis(point, axis)
    return point.x if axis == 0
    return point.y if axis == 1
    return point.z if axis == 2
  end
end

<main>:3: warning: already initialized constant Point::CLOSEST_POINTS_LIMIT
<main>:3: warning: previous definition of CLOSEST_POINTS_LIMIT was here


:attrbyAxis

In [2]:
class KdTreeNode
  attr_reader :value, :left, :right
  def initialize(value:, left:, right:)
    @value = value
    @left = left
    @right = right
  end
end

def buildKdTree(points, axis)
  return nil if points.nil? or points.empty?
  points = points.sort_by { |p| p.x } if axis == 0
  points = points.sort_by { |p| p.y } if axis == 1
  points = points.sort_by { |p| p.z } if axis == 2
  median = points.length / 2
  axis = (axis+1) % 3
  node = KdTreeNode.new( :value => points[median], 
    :left => buildKdTree(points[0...median], axis), 
    :right => buildKdTree(points[(median+1)...points.length], axis))
  return node
end

:buildKdTree

In [4]:
points = 10_000.times.map { Point.random }
puts

In [5]:
root = buildKdTree(points, 0)
puts

In [35]:
# Sanity check
points = 100_000.times.map { Point.random }
root = buildKdTree(points, 0)
tests_number = 100
pts = tests_number.times.map { Point.random }

pts.each do
  kd = target.cloest_points_2(node: root)
  brutal =  target.closest_points(points: points)
  if kd != brutal
    puts "The result of k-d tree is different from brutal"
    break
  end
end
puts

In [30]:
# performance test
points = 10_000_000.times.map { Point.random }
root = buildKdTree(points, 0)
tests_number = 1000
pts = tests_number.times.map { Point.random }

Benchmark.realtime do
  pts.each do |p|
    p.cloest_points_2(:node => root)
  end
end

0.4001950001111254

In [34]:
Benchmark.realtime do
  pts.each do |p|
    p.cloest_points_2(:node => root)
  end
end

0.6000040000071749

In [18]:
# while true do
target = Point.random
kd = target.cloest_points_2(node: root)
puts "kd: ", kd
puts kd.length
brutal =  target.closest_points(points: points)
puts "brutal: ", brutal
puts "miao"
puts kd[0] == brutal[0]
# break
# end

kd: 
[#<Point:0x007fdf910781b8 @x=0.9671325300189624, @y=0.4471754860753543, @z=0.42274186294619776, @min_points=[]>, #<Point:0x007fdf90b0e470 @x=0.9850651041780931, @y=0.49335247671332616, @z=0.47088853702434275, @min_points=[]>, #<Point:0x007fdf90b3c168 @x=0.956506126741443, @y=0.4849663552977518, @z=0.4249053113609341, @min_points=[]>, #<Point:0x007fdf90b077d8 @x=0.9750785472332648, @y=0.4269425870388869, @z=0.4678268110373254, @min_points=[]>, #<Point:0x007fdf91c11a38 @x=0.9847614659585888, @y=0.5046165346839105, @z=0.45026414690045935, @min_points=[]>, #<Point:0x007fdf91c104f8 @x=0.9875491108270079, @y=0.41628071659618726, @z=0.4366225209572844, @min_points=[]>, #<Point:0x007fdf91086150 @x=0.974340934306766, @y=0.40033582923678046, @z=0.46161814752509944, @min_points=[]>, #<Point:0x007fdf91ca87a8 @x=0.996782990753614, @y=0.44630824802357505, @z=0.3807317637734171, @min_points=[]>, #<Point:0x007fdf9089fe28 @x=0.9522189236492303, @y=0.5160860609124444, @z=0.4554393006306203, @min_po

In [14]:
kd[0]

[#<Point:0x007fdf909cef38 @x=0.32868596894707547, @y=0.2377247237229546, @z=0.6200282795900314, @min_points=[]>, 0.00043955401669878096]

In [19]:
brutal.pop()

#<Point:0x007fdf9199e1e8 @x=0.9795759891294815, @y=0.5088995103603892, @z=0.48934508590744485, @min_points=[]>

In [20]:
kd == brutal

false

In [28]:
tests_number = 1000
pts = tests_number.times.map { Point.random }

Benchmark.realtime do
  pts.each do |p|
    p.cloest_points_2(:node => root)
  end
end

0.26859800005331635

In [50]:
tests_number = 1
pts = tests_number.times.map { Point.random }

Benchmark.realtime do
  pts.each do |p|
     p.closest_points(points: points)
  end
end

33.13210499996785

In [190]:
tests_number = 10
points = tests_number.times.map { Point.random }
Benchmark.realtime do
  Point.random.closest_points(points: points)
end

3.700004890561104e-05

In [29]:
points = tests_number.times.map { Point.random }
Benchmark.realtime do
  Point.random.closest_points(points: points)
end

3.799994010478258e-05

In [233]:
a = [1]
a[0...0]

[]

In [273]:
stack = [3] * 10

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [182]:
a = [1,2,3]
puts a[a.length/2]
puts a[0...(a.length/2)]
puts a[(a.length/2+1)...a.length]

2
[]
[]


In [20]:
a = [[2,4],[3,2],[5,1],[9,1],[4,2]]
a = a.sort_by{ |x| x[1] }.first(10)

[[5, 1], [9, 1], [3, 2], [4, 2], [2, 4]]

In [17]:
a = []
a << 3

[3]

In [38]:
a = [1,2,3,4,5]
a.last

5

In [82]:
a = [nil, 3] * 10

[nil, 3, nil, 3, nil, 3, nil, 3, nil, 3, nil, 3, nil, 3, nil, 3, nil, 3, nil, 3]

In [103]:
a = [1,2,3,4,5,5,6,1,14,1,3,10]
a = a.sort()
b = [4]
a[0,9] += b

[1, 1, 1, 2, 3, 3, 4, 5, 5, 4]

In [23]:
a = [1,2,3,4,5]
a[0...-0]

[]

In [22]:
a = [1,2,3,4,5]
b = [1,2,3,4,5]
a == b

true